# AMUSE for NEMO users

In this notebook we will demonstrate some of the ways how [AMUSE](https://www.amusecode.org) and NEMO can exchange data. Since NEMO works normally in a shell, and AMUSE in a python environment, this notebook is thus somewhat limited in showing the full potential, since the cells need to be executed separately.





Make sure you start this notebook with NEMO preloaded in your shell, else the **%%bash** cells will (not easily) work. 

## 1. Check if NEMO is loaded

First, two sanity checks.  Is NEMO loaded. Is AMUSE available?

If not, you may need to know where NEMO was installed and do something like 
```
        source $HOME/NEMO/nemo/nemo_start.sh
or:
        [ -e ~/.nemorc ] && source ~/.nemorc && source $NEMO/nemo_start.sh
           
```
in each bash cell.   But be aware NEMO might also load another python not commensurable with AMUSE... YMMV, as they say.

In [ ]:
%%bash

nemo

**NOTE**: if the previous cell did not work, re-read the instruction before again, or else start the notebook after NEMO was loaded in the shell.

## 2. Check is AMUSE is present

Check if **AMUSE** can be imported, and where it's located.

In [ ]:
import amuse
print(amuse.__file__)

If amuse was not loaded, it needs to be pip installed. The `mknemo amuse` install method in NEMO is more for developers.  For this particular notebook we only need the framework and bhtree module, viz.
```
    pip install amuse-framework
    pip install amuse-bhtree
```
You can probably do that from within this notebook.

## 3. Make a Plummer model

Get the modules from AMUSE we need

In [ ]:
from amuse.units import nbody_system
from amuse.ic.plummer import new_plummer_model
from amuse.community.bhtree import Bhtree
from amuse.io import write_set_to_file


Create a small Plummer sphere and print the contents

In [ ]:
import numpy as np

np.random.seed(123)
stars = new_plummer_model(8)

print(stars)

## 4. Integrate a Plummer model

Set up an nbody integrator, and evolve the system

In [ ]:
gravity = Bhtree()

eps = 0.025 | nbody_system.length
tstop = 10  | nbody_system.time

gravity.parameters.opening_angle  = 0.75
gravity.parameters.epsilon_squared = eps * eps
gravity.parameters.dt_dia          = 1.0      | nbody_system.time
gravity.parameters.timestep        = 0.015625 | nbody_system.time       # 1/64

stars_in_gravity = gravity.particles.add_particles(stars)
gravity.evolve_model(tstop)



In [ ]:
# https://github.com/amusecode/amuse/issues/1121

# help(gravity.parameters)

Write the model to an ASCII table 

In [ ]:

write_set_to_file(stars_in_gravity, 'plummer.txt', format='txt')



## 5. Read the data back into NEMO

Back to NEMO, convert the ASCII table into a NEMO format

In [ ]:
%%bash

head -3 plummer.txt

rm -f plummer.snap
tabcols plummer.txt | tabtos - plummer.snap block1=mass,skip,vx,vy,vz,x,y,z nbody=$(tabcols plummer.txt | wc -l)
tsf plummer.snap


The default integration only saved the final time. One has to take some extra steps to do the equivalent of what in NEMO is common to also store the output with a given frequency.